In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [4]:
word_index = imdb.get_word_index()

reverse_word_index = {
    value: key for (key, value) in word_index.items()
}

In [5]:
model = load_model('imdb_rnn_model.h5')

2025-06-03 09:47:17.432619: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-06-03 09:47:17.432655: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-06-03 09:47:17.432665: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-06-03 09:47:17.432686: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-03 09:47:17.432705: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, 128)              │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,411,715 (5.39 MB)

 Trainable params: 1,411,713 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [12]:
def decode_review(text):
    """Decode the review text from integers to words."""
    return ' '.join(
        reverse_word_index.get(i - 3, '?') for i in text
    )

def preprocess_review(review):
    """Preprocess the review text for prediction."""
    # Convert the review to integers
    encoded_rev = [word_index.get(word, 2) + 3 for word in review.lower().split()]
    # Pad the sequence to ensure it has the same length as the training data
    padded = sequence.pad_sequences([encoded_rev], maxlen=500)
    return padded

In [14]:
def predict_review(review):
    predictInput = preprocess_review(review)
    prediction = model.predict(predictInput)
    sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
    return sentiment, prediction[0][0]

In [22]:
review = "The movie was very bad and boring."
sentiment, score = predict_review(review)
print(f"Sentiment: {sentiment}, Confidence Score: {score:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Sentiment: negative, Confidence Score: 0.27
